## This Notebook is for testing testing code and modules

In [1]:
import logging
logging.basicConfig(filename="debugging_nb.log",
                    level=logging.DEBUG,
                    format="%(asctime)s:%(levelname)s:%(message)s"
                   )

debug_nb = logging.getLogger("Debugging_nb")
#downSampler_logger = logging.getLogger("downSampler")
#downSampler_logger.debug("Import libraries loaded")
debug_nb.debug("Start of new run"+"-"*50)

In [1]:
import os
import sys
from pathlib import Path

%load_ext autoreload
%autoreload 2

module_path = os.path.abspath(os.path.join('..')) # add the projects src code directory to the sys path to allow finding our module
if module_path not in sys.path:
    sys.path.append(module_path+"\\src\\data")
    #sys.path.insert(0, module_path+"\\src\\data")
    

import downSampler # load pipeline module

import json
import h5py
import gdal
gdal.UseExceptions()
import itertools
import pystac
import geojson as gj
import numpy as np
import tqdm

In [3]:
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



## Read in Data files

In [2]:
input_data_path = Path(os.getcwd()).parents[0] / 'data' / 'pipeline_input' # path to input data files
output_data_path = Path(os.getcwd()).parents[0] / 'data' / 'pipeline_output' # path to input data files
input_file_dict  = downSampler.find_files(input_data_path)
#input_file_dict
output_file_dict = downSampler.find_files(output_data_path)
#output_file_dict

Finding data files...
Finding data files...


In [6]:
in_keys_ls = list(input_file_dict.keys())
in_keys_ls
# next(iter(input_file_dict)) alternative way
downSampler.h5dump(input_file_dict[in_keys_ls[0]])

print("-"*75)
out_keys_ls = list(output_file_dict.keys())
out_keys_ls
# next(iter(input_file_dict)) alternative way
downSampler.h5dump(output_file_dict[out_keys_ls[0]])

	 - ABBY : <HDF5 group "/ABBY" (1 members)>
		 - Reflectance : <HDF5 group "/ABBY/Reflectance" (2 members)>
			 - Metadata : <HDF5 group "/ABBY/Reflectance/Metadata" (7 members)>
				 - Ancillary_Imagery : <HDF5 group "/ABBY/Reflectance/Metadata/Ancillary_Imagery" (14 members)>
					 - Aerosol_Optical_Depth : <HDF5 dataset "Aerosol_Optical_Depth": shape (1000, 1000), type "<i2">
							 - Band_Names : b'AOT (aerosol optical thickness at 550 nm)*1000'
							 - Data_Ignore_Value : -9999.0
							 - Description : b'Aerosol Optical Depth at 500 nm.'
							 - Scale_Factor : 1000.0
							 - Units : b'Aerosol Optical Depth at 500 nm.'
					 - Aspect : <HDF5 dataset "Aspect": shape (1000, 1000), type "<f4">
							 - Data_Ignore_Value : -9999.0
							 - Description : b'Aspect used as input to ATCOR'
							 - Dimension_Labels : b'-'
							 - Scale_Factor : 1.0
							 - Units : b'degrees'
					 - Cast_Shadow : <HDF5 dataset "Cast_Shadow": shape (1000, 1000), type "|u1">
							 - Data_Ign

### Read in one NEON hdf5 file

In [7]:
input_file_dict[in_keys_ls[0]]

'C:\\Users\\Chris\\Documents\\SideProjects_C\\hyperspectral-project\\data\\pipeline_input\\NEON.D16.ABBY.DP3.30006.001.2019-07.basic.20210211T072435Z.RELEASE-2021\\NEON_D16_ABBY_DP3_552000_5071000_reflectance.h5'

In [8]:
refl_array, wavelength_array, FWHM_array, metadata_dict = downSampler.h5data2array(input_file_dict[in_keys_ls[0]])

### Checks on reflectance array just read in from hdf5 file

In [9]:
refl_array

array([[[0.0092, 0.0187, 0.0168, ..., 0.0184, 0.025 , 0.0483],
        [0.0108, 0.0179, 0.0136, ..., 0.0089, 0.0307, 0.0218],
        [0.0103, 0.0161, 0.016 , ..., 0.0216, 0.0352, 0.048 ],
        ...,
        [0.0115, 0.0211, 0.0224, ..., 0.0071, 0.0316, 0.088 ],
        [0.012 , 0.0229, 0.0215, ..., 0.0089, 0.0312, 0.0905],
        [0.0102, 0.0239, 0.0195, ..., 0.0425, 0.0499, 0.1174]],

       [[0.0126, 0.0198, 0.0178, ..., 0.0121, 0.0189, 0.0293],
        [0.0129, 0.0224, 0.0209, ..., 0.0333, 0.0183, 0.0628],
        [0.0141, 0.0176, 0.0191, ..., 0.0164, 0.0042, 0.0336],
        ...,
        [0.0158, 0.0272, 0.0239, ..., 0.0175, 0.0738, 0.0626],
        [0.0114, 0.0248, 0.0231, ..., 0.0324, 0.0851, 0.1389],
        [0.0132, 0.018 , 0.014 , ..., 0.0076, 0.0806, 0.093 ]],

       [[0.0122, 0.0209, 0.0247, ..., 0.0182, 0.0415, 0.0608],
        [0.0178, 0.0227, 0.0226, ..., 0.0253, 0.0282, 0.0371],
        [0.0125, 0.0175, 0.0242, ..., 0.0123, 0.0478, 0.0484],
        ...,
        [0.0

In [10]:
# resampled input image
for band in range(50): #refl_array.shape[2]):
    print('Band: ', band)
    print('Max: ', refl_array[:,:,band].max())
    print('Min: ', refl_array[:,:,band].min())
    print('Avg.: ', np.average(refl_array[:,:,band]))
    print('Std.: ', np.std(refl_array[:,:,band]))

Band:  0
Max:  0.135
Min:  0.0
Avg.:  0.015324575500000012
Std.:  0.006813664253160685
Band:  1
Max:  1.0
Min:  0.0
Avg.:  0.025318361400000003
Std.:  0.012755824928987934
Band:  2
Max:  0.2277
Min:  0.0
Avg.:  0.02338669219999999
Std.:  0.009940997643217663
Band:  3
Max:  0.2937
Min:  0.0
Avg.:  0.02372840600000001
Std.:  0.010624673715421288
Band:  4
Max:  0.3254
Min:  0.0015
Avg.:  0.018677119800000003
Std.:  0.009804547171412251
Band:  5
Max:  0.3828
Min:  0.0003
Avg.:  0.018358685000000007
Std.:  0.010041865987493313
Band:  6
Max:  0.4534
Min:  0.0
Avg.:  0.019997341600000006
Std.:  0.010975875884543766
Band:  7
Max:  0.4748
Min:  0.0
Avg.:  0.018837172
Std.:  0.010874770458378239
Band:  8
Max:  0.5108
Min:  0.0019
Avg.:  0.019118510300000002
Std.:  0.011229915740502857
Band:  9
Max:  0.5257
Min:  0.0022
Avg.:  0.019451222400000003
Std.:  0.01154634859017076
Band:  10
Max:  0.5548
Min:  0.0028
Avg.:  0.02094472640000001
Std.:  0.01241219039288163
Band:  11
Max:  0.5538
Min:  0.002


It appears that the input file and functionality are working correctly at this point, based on the fact that all the bands are read in and they are unique.

-----------------------------------------------

### Now to call the resampling function and observe the behaviour of the returned arrays bands

In [28]:
# parameter setup:
data_dir_path = Path(os.getcwd()).parents[0] / 'data' / 'pipeline_input' # path to input data files
output_data_path = Path(os.getcwd()).parents[0] / 'data' / 'pipeline_output' # path to input data files
desired_GSD = 5 # GSD in m
desired_band_centres = [0.505, 0.526, 0.544, 0.565, 0.586, 0.606, 0.626, 0.646, 0.665, 0.682, 0.699, 0.715, 0.730, 0.745, 0.762, 0.779, 0.787, 0.804]
desired_band_centres_um = desired_band_centres # THIS IS TO TEST/SHOW THE BUG - Don't actually use it

desired_band_centres = np.array(desired_band_centres)*1000 # this is the fix line
print("Parameters set!")

Parameters set!


In [38]:
print(desired_band_centres) # look at input band centres

[505. 526. 544. 565. 586. 606. 626. 646. 665. 682. 699. 715. 730. 745.
 762. 779. 787. 804.]


In [39]:
desired_band_centres_um

[0.505,
 0.526,
 0.544,
 0.565,
 0.586,
 0.606,
 0.626,
 0.646,
 0.665,
 0.682,
 0.699,
 0.715,
 0.73,
 0.745,
 0.762,
 0.779,
 0.787,
 0.804]

In [37]:
downSampler.reband_spectral_array(refl_array[1,0,:], wavelength_array, desired_band_centres_um)

array([0.0126, 0.0126, 0.0126, 0.0126, 0.0126, 0.0126, 0.0126, 0.0126,
       0.0126, 0.0126, 0.0126, 0.0126, 0.0126, 0.0126, 0.0126, 0.0126,
       0.0126, 0.0126])

In [35]:
#wavelength_array
refl_array

array([[[0.0092, 0.0187, 0.0168, ..., 0.0184, 0.025 , 0.0483],
        [0.0108, 0.0179, 0.0136, ..., 0.0089, 0.0307, 0.0218],
        [0.0103, 0.0161, 0.016 , ..., 0.0216, 0.0352, 0.048 ],
        ...,
        [0.0115, 0.0211, 0.0224, ..., 0.0071, 0.0316, 0.088 ],
        [0.012 , 0.0229, 0.0215, ..., 0.0089, 0.0312, 0.0905],
        [0.0102, 0.0239, 0.0195, ..., 0.0425, 0.0499, 0.1174]],

       [[0.0126, 0.0198, 0.0178, ..., 0.0121, 0.0189, 0.0293],
        [0.0129, 0.0224, 0.0209, ..., 0.0333, 0.0183, 0.0628],
        [0.0141, 0.0176, 0.0191, ..., 0.0164, 0.0042, 0.0336],
        ...,
        [0.0158, 0.0272, 0.0239, ..., 0.0175, 0.0738, 0.0626],
        [0.0114, 0.0248, 0.0231, ..., 0.0324, 0.0851, 0.1389],
        [0.0132, 0.018 , 0.014 , ..., 0.0076, 0.0806, 0.093 ]],

       [[0.0122, 0.0209, 0.0247, ..., 0.0182, 0.0415, 0.0608],
        [0.0178, 0.0227, 0.0226, ..., 0.0253, 0.0282, 0.0371],
        [0.0125, 0.0175, 0.0242, ..., 0.0123, 0.0478, 0.0484],
        ...,
        [0.0

In [30]:

desired_band_centres

array([505., 526., 544., 565., 586., 606., 626., 646., 665., 682., 699.,
       715., 730., 745., 762., 779., 787., 804.])

In [24]:
refl_array

array([[[0.0092, 0.0187, 0.0168, ..., 0.0184, 0.025 , 0.0483],
        [0.0108, 0.0179, 0.0136, ..., 0.0089, 0.0307, 0.0218],
        [0.0103, 0.0161, 0.016 , ..., 0.0216, 0.0352, 0.048 ],
        ...,
        [0.0115, 0.0211, 0.0224, ..., 0.0071, 0.0316, 0.088 ],
        [0.012 , 0.0229, 0.0215, ..., 0.0089, 0.0312, 0.0905],
        [0.0102, 0.0239, 0.0195, ..., 0.0425, 0.0499, 0.1174]],

       [[0.0126, 0.0198, 0.0178, ..., 0.0121, 0.0189, 0.0293],
        [0.0129, 0.0224, 0.0209, ..., 0.0333, 0.0183, 0.0628],
        [0.0141, 0.0176, 0.0191, ..., 0.0164, 0.0042, 0.0336],
        ...,
        [0.0158, 0.0272, 0.0239, ..., 0.0175, 0.0738, 0.0626],
        [0.0114, 0.0248, 0.0231, ..., 0.0324, 0.0851, 0.1389],
        [0.0132, 0.018 , 0.014 , ..., 0.0076, 0.0806, 0.093 ]],

       [[0.0122, 0.0209, 0.0247, ..., 0.0182, 0.0415, 0.0608],
        [0.0178, 0.0227, 0.0226, ..., 0.0253, 0.0282, 0.0371],
        [0.0125, 0.0175, 0.0242, ..., 0.0123, 0.0478, 0.0484],
        ...,
        [0.0

In [23]:
wavelength_array

array([ 381.598 ,  386.6077,  391.6175,  396.6273,  401.637 ,  406.6468,
        411.6566,  416.6663,  421.6761,  426.6858,  431.6956,  436.7054,
        441.7151,  446.7249,  451.7347,  456.7444,  461.7542,  466.764 ,
        471.7737,  476.7835,  481.7933,  486.803 ,  491.8128,  496.8226,
        501.8323,  506.8421,  511.8519,  516.8616,  521.8714,  526.8812,
        531.8909,  536.9007,  541.9105,  546.9202,  551.93  ,  556.9398,
        561.9495,  566.9593,  571.9691,  576.9788,  581.9886,  586.9984,
        592.0081,  597.0179,  602.0277,  607.0374,  612.0472,  617.057 ,
        622.0667,  627.0765,  632.0863,  637.096 ,  642.1058,  647.1156,
        652.1253,  657.1351,  662.1449,  667.1546,  672.1644,  677.1742,
        682.1839,  687.1937,  692.2035,  697.2132,  702.223 ,  707.2328,
        712.2425,  717.2523,  722.2621,  727.2718,  732.2816,  737.2914,
        742.3011,  747.3109,  752.3207,  757.3304,  762.3402,  767.35  ,
        772.3597,  777.3695,  782.3793,  787.389 , 

In [17]:
resamp_refl_array = downSampler.downSample_reband_array(refl_array, metadata_dict['Spatial_Resolution_X_Y'][0], desired_GSD, wavelength_array, desired_band_centres) # downsample

resamp_metadata_dict = metadata_dict.copy()
resamp_metadata_dict['Spatial_Resolution_X_Y'] = [float(desired_GSD), float(desired_GSD)] # adjust resolution metadata to reflect downsampling
print("Downsampling Complete!")

Downsampling Complete!


### Checks on re-sampled array just processed from refl_array

In [18]:
resamp_refl_array

array([[[0.0097, 0.0194, 0.0263, ..., 0.1856, 0.1866, 0.1903],
        [0.0109, 0.0213, 0.0287, ..., 0.1959, 0.1969, 0.2006],
        [0.012 , 0.0232, 0.0311, ..., 0.2061, 0.2072, 0.2108],
        ...,
        [0.0142, 0.0283, 0.0385, ..., 0.3148, 0.3153, 0.3206],
        [0.0128, 0.0248, 0.0336, ..., 0.2702, 0.2706, 0.2753],
        [0.0114, 0.0213, 0.0286, ..., 0.2256, 0.2259, 0.2301]],

       [[0.0093, 0.0183, 0.0247, ..., 0.1751, 0.1761, 0.1796],
        [0.0107, 0.0206, 0.0276, ..., 0.1874, 0.1885, 0.192 ],
        [0.0121, 0.0228, 0.0304, ..., 0.1998, 0.2009, 0.2043],
        ...,
        [0.014 , 0.028 , 0.0381, ..., 0.3123, 0.3129, 0.3183],
        [0.0122, 0.024 , 0.0325, ..., 0.2648, 0.2653, 0.2701],
        [0.0104, 0.0199, 0.027 , ..., 0.2173, 0.2177, 0.2219]],

       [[0.0088, 0.0172, 0.0231, ..., 0.1647, 0.1656, 0.169 ],
        [0.0105, 0.0199, 0.0264, ..., 0.179 , 0.1801, 0.1834],
        [0.0122, 0.0225, 0.0298, ..., 0.1934, 0.1946, 0.1977],
        ...,
        [0.0

In [21]:
# testing & error checking

def array_stats(input_array):
    print("Mean reflectance: "'{number:.{digits}f}'.format(number=input_array.mean(), digits=3))
    print("Std Dev.: "'{number:.{digits}f}'.format(number=input_array.std(), digits=3))
    print("Min reflectance: "'{number:.{digits}f}'.format(number=input_array.min(), digits=3))
    print("Max reflectance: "'{number:.{digits}f}'.format(number=input_array.max(), digits=3))
    print("-"*35)
    
array_stats(resamp_refl_array[:,:,0])
array_stats(resamp_refl_array[:,:,6])
array_stats(resamp_refl_array[:,:,13])
print('-'*50)
print(resamp_refl_array[100:110,100:110,0])
print('-'*50)
print(resamp_refl_array[100:110,100:110,6])
print('-'*50)
print(resamp_refl_array[100:110,100:110,13])

Mean reflectance: 0.014
Std Dev.: 0.011
Min reflectance: 0.000
Max reflectance: 0.312
-----------------------------------
Mean reflectance: 0.022
Std Dev.: 0.017
Min reflectance: 0.000
Max reflectance: 0.340
-----------------------------------
Mean reflectance: 0.245
Std Dev.: 0.073
Min reflectance: 0.018
Max reflectance: 0.617
-----------------------------------
--------------------------------------------------
[[0.0082 0.006  0.0037 0.0049 0.0061 0.0073 0.0085 0.0097 0.0087 0.0077]
 [0.0082 0.0056 0.003  0.0046 0.0063 0.008  0.0097 0.0114 0.0101 0.0088]
 [0.0081 0.0052 0.0022 0.0044 0.0065 0.0087 0.0109 0.0131 0.0115 0.01  ]
 [0.0073 0.0047 0.0021 0.0039 0.0057 0.0075 0.0093 0.0111 0.0098 0.0085]
 [0.0065 0.0042 0.002  0.0034 0.0048 0.0062 0.0076 0.0091 0.008  0.007 ]
 [0.0056 0.0038 0.0019 0.003  0.004  0.005  0.006  0.0071 0.0063 0.0055]
 [0.0048 0.0033 0.0018 0.0025 0.0031 0.0038 0.0044 0.005  0.0046 0.0041]
 [0.004  0.0028 0.0017 0.002  0.0023 0.0025 0.0028 0.003  0.0028 0.0026]

In [22]:
# resampled input image
for band in range(resamp_refl_array.shape[2]):
    print('Band: ', band)
    print('Max: ', resamp_refl_array[:,:,band].max())
    print('Min: ', resamp_refl_array[:,:,band].min())
    print('Avg.: ', np.average(resamp_refl_array[:,:,band]))
    print('Std.: ', np.std(resamp_refl_array[:,:,band]))

Band:  0
Max:  0.3121
Min:  0.0
Avg.:  0.014048242599999998
Std.:  0.010983405689108699
Band:  1
Max:  0.3171
Min:  0.0001
Avg.:  0.026515900599999996
Std.:  0.01293387860121316
Band:  2
Max:  0.3321
Min:  0.0004
Avg.:  0.036145855500000004
Std.:  0.01541627140534052
Band:  3
Max:  0.3348
Min:  0.0001
Avg.:  0.035302929000000004
Std.:  0.016083720249399983
Band:  4
Max:  0.3385
Min:  0.0
Avg.:  0.027212828099999996
Std.:  0.015823635062458007
Band:  5
Max:  0.341
Min:  0.0
Avg.:  0.025067797399999994
Std.:  0.016358443255168054
Band:  6
Max:  0.3403
Min:  0.0
Avg.:  0.021706620100000004
Std.:  0.01666816769246926
Band:  7
Max:  0.3428
Min:  0.0
Avg.:  0.018649912200000007
Std.:  0.01688462219039239
Band:  8
Max:  0.339
Min:  0.0
Avg.:  0.012510422899999998
Std.:  0.016387416121315636
Band:  9
Max:  0.3315
Min:  0.0
Avg.:  0.0129249105
Std.:  0.01672133468049096
Band:  10
Max:  0.34
Min:  0.0
Avg.:  0.039913677300000006
Std.:  0.021836957121390898
Band:  11
Max:  0.3339
Min:  0.003
Avg.

### Read in file and check bands

In [25]:
from osgeo import gdal
import numpy as np
temp_path = r'C:\Users\Chris\Documents\SideProjects_C\hyperspectral-project\data\Skywatch\catalog\WYVERN_DS_20210520T094044_C8ovySO_band_001.tif'

gtif = gdal.Open( temp_path )
band = gtif.GetRasterBand(1)
bandArray1 = band.ReadAsArray()
print(bandArray1)
print(np.max(bandArray1))
print(np.min(bandArray1))
print(np.mean(bandArray1))
gtif = None

[[0.0283 0.026  0.0236 ... 0.0682 0.0677 0.0673]
 [0.0295 0.0266 0.0237 ... 0.0679 0.0677 0.0676]
 [0.0308 0.0273 0.0238 ... 0.0677 0.0678 0.0678]
 ...
 [0.0279 0.0272 0.0264 ... 0.0162 0.02   0.0237]
 [0.0278 0.0274 0.027  ... 0.0176 0.0217 0.0258]
 [0.0276 0.0276 0.0277 ... 0.0189 0.0234 0.028 ]]
0.3691
0.0048
0.04041837


In [27]:
from osgeo import gdal
import numpy as np
temp_path = r'C:\Users\Chris\Documents\SideProjects_C\hyperspectral-project\data\Skywatch\catalog\WYVERN_DS_20210520T094044_C8ovySO_band_007.tif'

gtif = gdal.Open( temp_path )
band = gtif.GetRasterBand(1)
bandArray7 = band.ReadAsArray()
print(bandArray7)
print(np.max(bandArray7))
print(np.min(bandArray7))
print(np.mean(bandArray7))
gtif = None

[[0.0391 0.035  0.0309 ... 0.0935 0.0929 0.0923]
 [0.041  0.036  0.031  ... 0.0936 0.0934 0.0932]
 [0.0429 0.037  0.0311 ... 0.0937 0.0939 0.0941]
 ...
 [0.0392 0.0381 0.0369 ... 0.0193 0.0249 0.0305]
 [0.0391 0.0385 0.0379 ... 0.0216 0.0278 0.0339]
 [0.039  0.039  0.039  ... 0.0238 0.0306 0.0374]]
0.4042
0.0034
0.057678293


## Array to hdf5 raster layer image:

In [29]:
# save to raster hdf5
refl_arrayname_output =  Path(output_data_path / 'splitbands_output_Debugging_Test.h5')
band_width_array = downSampler.band_widths(desired_band_centres)
downSampler.array2h5raster(resamp_refl_array, desired_band_centres, band_width_array, resamp_metadata_dict, refl_arrayname_output)

Band_001
Band_002
Band_003
Band_004
Band_005
Band_006
Band_007
Band_008
Band_009
Band_010
Band_011
Band_012
Band_013
Band_014
Band_015
Band_016
Band_017
Band_018


## Array to Geotiff raster layer image:

In [30]:
# test it out and check outputs in QGIS
downSampler.array2gtiff_raster(resamp_refl_array, desired_band_centres, band_width_array, resamp_metadata_dict, 1, 1, output_data_path, "Debugging_Test_Geotiff")

In [3]:
input_file_dict

{'NEON_D16_ABBY_DP3_552000_5071000_reflectance.h5': 'C:\\Users\\Chris\\Documents\\SideProjects_C\\hyperspectral-project\\data\\pipeline_input\\NEON.D16.ABBY.DP3.30006.001.2019-07.basic.20210211T072435Z.RELEASE-2021\\NEON_D16_ABBY_DP3_552000_5071000_reflectance.h5',
 'NEON_D16_ABBY_DP3_552000_5072000_reflectance.h5': 'C:\\Users\\Chris\\Documents\\SideProjects_C\\hyperspectral-project\\data\\pipeline_input\\NEON.D16.ABBY.DP3.30006.001.2019-07.basic.20210211T072435Z.RELEASE-2021\\NEON_D16_ABBY_DP3_552000_5072000_reflectance.h5',
 'NEON_D16_ABBY_DP3_552000_5073000_reflectance.h5': 'C:\\Users\\Chris\\Documents\\SideProjects_C\\hyperspectral-project\\data\\pipeline_input\\NEON.D16.ABBY.DP3.30006.001.2019-07.basic.20210211T072435Z.RELEASE-2021\\NEON_D16_ABBY_DP3_552000_5073000_reflectance.h5',
 'NEON_D16_ABBY_DP3_552000_5074000_reflectance.h5': 'C:\\Users\\Chris\\Documents\\SideProjects_C\\hyperspectral-project\\data\\pipeline_input\\NEON.D16.ABBY.DP3.30006.001.2019-07.basic.20210211T072435Z.

In [9]:
import time
for file_name, file_path in tqdm.tqdm(input_file_dict.items(), desc = "Processing"):
    time.sleep(1)

Processing: 100%|████████████████████████████████████████████████████████████████████████| 6/6 [00:06<00:00,  1.01s/it]


## -------------------------------------- WIP AFTER THIS POINT --------------------------------------

In [ ]:
# test_ls = np.array([
#                    [[1,2,3,4],
#                     [2,3,4,5],
#                     [3,4,5,6],
#                     [4,5,6,7]],
    
#                    [[11,12,13,14],
#                     [12,13,14,15],
#                     [13,14,15,16],
#                     [14,15,16,17]],
    
#                    [[21,22,23,24],
#                     [22,23,24,25],
#                     [23,24,25,26],
#                     [24,25,26,27]],
    
#                    [[31,32,33,34],
#                     [32,33,34,35],
#                     [33,34,35,36],
#                     [34,35,36,37]],
    
#                     ])
# test_ls
# test_ls.shape
# test_ls[2,2,2]